# Một bài toán trích chọn thông tin mới: trích xuất thông tin về các khoản đầu tư và startup từ các bài báo kinh tế.

1. Để dataset thành 1 file chứa tập train và tập test.

In [1]:
import json
import random

# Danh sách các startup, nhà đầu tư, và lĩnh vực
startups = [
    "VinAI", "Tiki", "Grab Việt Nam", "GoJek", "Being",
    "Sendo", "Lazada", "MoMo", "VNG", "Loft"
]

investors = [
    "Softbank", "GIC", "Sequoia Capital", "IDG Ventures",
    "500 Startups", "Vertex Ventures", "Grab Ventures",
    "East Ventures", "Jungle Ventures"
]

sectors = [
    "Công nghệ thông tin", "Thương mại điện tử",
    "Fintech", "Logistics", "Trí tuệ nhân tạo"
]

def generate_investment_dataset(num_samples=100):
    dataset = {
        "train": [],
        "test": []
    }

    # Hàm sinh câu miêu tả đầu tư
    def generate_investment_description():
        startup = random.choice(startups)
        investor = random.choice(investors)
        sector = random.choice(sectors)
        amount = round(random.uniform(0.5, 100), 2)

        templates = [
            f"{startup}, một startup hoạt động trong lĩnh vực {sector}, vừa nhận được khoản đầu tư {amount} triệu USD từ {investor}.",
            f"{investor} đã rót {amount} triệu USD vào {startup} - startup triển vọng trong ngành {sector}.",
            f"Mới đây, {startup} đã gọi vốn thành công {amount} triệu USD từ quỹ đầu tư {investor} trong lĩnh vực {sector}."
        ]

        return {
            "text": random.choice(templates),
            "entities": {
                "startup": startup,
                "investor": investor,
                "sector": sector,
                "investment_amount": amount
            }
        }

    # Sinh dữ liệu huấn luyện
    for _ in range(int(num_samples * 0.8)):
        dataset["train"].append(generate_investment_description())

    # Sinh dữ liệu test
    for _ in range(int(num_samples * 0.2)):
        dataset["test"].append(generate_investment_description())

    return dataset

# Sinh dữ liệu và lưu ra file
investment_dataset = generate_investment_dataset()

with open('startup_investment_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(investment_dataset, f, ensure_ascii=False, indent=2)

# In ra một số ví dụ mẫu
print("Mẫu dữ liệu huấn luyện:")
for sample in investment_dataset["train"][:3]:
    print(json.dumps(sample, ensure_ascii=False, indent=2))


Mẫu dữ liệu huấn luyện:
{
  "text": "Mới đây, GoJek đã gọi vốn thành công 10.71 triệu USD từ quỹ đầu tư Sequoia Capital trong lĩnh vực Công nghệ thông tin.",
  "entities": {
    "startup": "GoJek",
    "investor": "Sequoia Capital",
    "sector": "Công nghệ thông tin",
    "investment_amount": 10.71
  }
}
{
  "text": "VNG, một startup hoạt động trong lĩnh vực Công nghệ thông tin, vừa nhận được khoản đầu tư 42.44 triệu USD từ Softbank.",
  "entities": {
    "startup": "VNG",
    "investor": "Softbank",
    "sector": "Công nghệ thông tin",
    "investment_amount": 42.44
  }
}
{
  "text": "Grab Ventures đã rót 97.66 triệu USD vào Being - startup triển vọng trong ngành Thương mại điện tử.",
  "entities": {
    "startup": "Being",
    "investor": "Grab Ventures",
    "sector": "Thương mại điện tử",
    "investment_amount": 97.66
  }
}


In [2]:
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder

class StartupInvestmentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, label_encoder):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_encoder = label_encoder

        self.labels = ['O', 'B-STARTUP', 'I-STARTUP',
                       'B-INVESTOR', 'I-INVESTOR',
                       'B-SECTOR', 'I-SECTOR',
                       'B-AMOUNT', 'I-AMOUNT']

        self.label_map = {label: idx for idx, label in enumerate(self.labels)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']

        # Tokenize và căn chỉnh nhãn
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        # Khởi tạo nhãn ban đầu
        labels = np.zeros(self.max_len, dtype=int)
        labels[:] = self.label_map['O']  # Mặc định là Outside

        # Trích xuất các thực thể
        entities = item['entities']

        # Ánh xạ các nhãn cho từng thực thể
        entity_mappings = [
            ('startup', 'STARTUP'),
            ('investor', 'INVESTOR'),
            ('sector', 'SECTOR'),
            ('investment_amount', 'AMOUNT')
        ]

        # Tìm vị trí của từng thực thể trong văn bản
        for entity_key, entity_type in entity_mappings:
            # Chuyển đổi giá trị thành chuỗi
            entity_value = str(entities[entity_key])
            start_idx = text.find(entity_value)

            if start_idx != -1:
                # Tokenize văn bản để tìm token tương ứng
                tokens = self.tokenizer.encode(text, add_special_tokens=False)
                entity_tokens = self.tokenizer.encode(entity_value, add_special_tokens=False)

                # Đánh dấu nhãn cho từng token của thực thể
                for token_idx, token in enumerate(tokens):
                    if tokens[token_idx:token_idx+len(entity_tokens)] == entity_tokens:
                        # Gán nhãn BIO
                        labels[token_idx] = self.label_map[f'B-{entity_type}']
                        for i in range(1, len(entity_tokens)):
                            labels[token_idx+i] = self.label_map[f'I-{entity_type}']

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels)
        }

def compute_metrics(p):
    """Tính toán độ đo Precision, Recall, F1-score"""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Loại bỏ các tokens padding
    mask = labels != -100

    true_labels = labels[mask]
    pred_labels = predictions[mask]

    # Tính độ đo cho từng nhãn
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, pred_labels, average='weighted'
    )

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

def load_dataset(file_path):
    """Tải dữ liệu từ file JSON"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def main():
    # Cấu hình
    model_name = 'vinai/phobert-base'  # Mô hình ngôn ngữ tiếng Việt
    max_len = 128
    batch_size = 16
    num_epochs = 5
    learning_rate = 2e-5

    # Tải dữ liệu
    dataset = load_dataset('startup_investment_dataset.json')

    # Tokenizer và mô hình
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(['O', 'B-STARTUP', 'I-STARTUP',
                        'B-INVESTOR', 'I-INVESTOR',
                        'B-SECTOR', 'I-SECTOR',
                        'B-AMOUNT', 'I-AMOUNT'])
    )

    # Tạo dataset
    label_encoder = LabelEncoder()
    train_dataset = StartupInvestmentDataset(
        dataset['train'],
        tokenizer,
        max_len,
        label_encoder
    )
    test_dataset = StartupInvestmentDataset(
        dataset['test'],
        tokenizer,
        max_len,
        label_encoder
    )

    # Cấu hình huấn luyện
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True
    )

    # Khởi tạo Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Huấn luyện mô hình
    trainer.train()

    # Đánh giá trên tập test
    eval_results = trainer.evaluate()
    print("Kết quả đánh giá:")
    print(f"Precision: {eval_results['eval_precision']:.4f}")
    print(f"Recall: {eval_results['eval_recall']:.4f}")
    print(f"F1-score: {eval_results['eval_f1_score']:.4f}")

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Score
1,No log,1.035398,0.925259,0.943750,0.932587
2,No log,0.486801,0.915161,0.956641,0.935441
3,No log,0.308102,0.915161,0.956641,0.935441
4,No log,0.262056,0.915161,0.956641,0.935441
5,No log,0.250498,0.915161,0.956641,0.935441


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

Kết quả đánh giá:
Precision: 0.9152
Recall: 0.9566
F1-score: 0.9354


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
